In [1]:
%matplotlib inline
import torch
import torch.nn.functional as F
import os, sys
from pathlib import Path
from TCN.mnist_pixel.utils import data_generator
from TCN.mnist_pixel.model import TCN
import numpy as np
import matplotlib.pyplot as plt
import pickle
from IPython import display
import ipywidgets as widgets
from PIL import Image
import io
import cv2
import time

In [2]:
model_path = Path('./TCN/mnist_pixel/models')
data_path = Path('./TCN/mnist_pixel/data/mnist')
model_name = 'aug_k7l6'
batch_size = 1
in_channels = 1
n_classes = 10

args = pickle.load(open(model_path / (model_name+'_args.pkl'), 'rb'))
print(args)
channel_sizes = [args.nhid] * args.levels

print(model_path / (model_name+'.pt'))

_, test_loader = data_generator(data_path, batch_size)
model = TCN(in_channels, n_classes, channel_sizes, kernel_size=args.ksize, inference=True)
model.load_state_dict(torch.load(model_path / (model_name+'.pt')), strict=False)
model.eval()

print(model.receptive_field)

if torch.cuda.is_available():
    model.cuda()
    
# model.set_fast_inference(batch_size)

Namespace(batch_size=64, clip=-1, cuda=True, dropout=0.05, epochs=50, ksize=7, levels=6, log_interval=100, lr=0.002, modelname='aug_k7l6', nhid=25, optim='Adam', permute=False, savedir=PosixPath('models'), savemodel=True, seed=-1, seq_augment=True)
TCN/mnist_pixel/models/aug_k7l6.pt
757


In [3]:
state_dict = torch.load(model_path / (model_name+'.pt'))
state_dict['tcn.network.0.net.0.weight_v']

tensor([[[-1.2557e-02,  2.7431e-01, -2.0780e-01, -4.0843e-01,  1.9649e-02,
           3.0148e-01,  5.2690e-02]],

        [[-3.0372e-01,  3.5627e-01,  2.3125e-01, -2.2784e-01, -1.7516e-01,
           3.6436e-02,  1.3082e-01]],

        [[ 3.3507e-01,  2.4562e-01, -2.1285e-01, -2.3298e-01, -4.3068e-01,
          -4.4790e-01,  1.1296e-01]],

        [[-4.5155e-01,  5.3474e-02,  6.6075e-02,  1.7959e-01,  6.8344e-02,
           6.2613e-02, -5.0894e-01]],

        [[ 1.0765e-02, -3.1726e-02,  2.7537e-01, -2.4272e-01, -3.9725e-01,
           3.7962e-01,  2.0172e-02]],

        [[-3.6972e-01, -1.4363e-01, -1.6847e-01, -1.9065e-01, -2.2254e-01,
          -1.5158e-01,  2.1970e-01]],

        [[-9.4216e-02,  1.9399e-01,  2.6476e-01, -4.0286e-01, -3.4270e-01,
           4.5129e-01,  2.3932e-02]],

        [[ 3.1543e-01, -1.4889e-01, -3.6360e-01,  6.5515e-02,  2.6093e-01,
          -2.7408e-01,  2.9647e-01]],

        [[-1.9951e-01,  1.5341e-01,  3.5835e-01, -1.6046e-01, -3.5035e-01,
           1.

In [10]:
state_dict.keys()

odict_keys(['tcn.network.0.conv1.bias', 'tcn.network.0.conv1.weight_g', 'tcn.network.0.conv1.weight_v', 'tcn.network.0.conv2.bias', 'tcn.network.0.conv2.weight_g', 'tcn.network.0.conv2.weight_v', 'tcn.network.0.net.0.bias', 'tcn.network.0.net.0.weight_g', 'tcn.network.0.net.0.weight_v', 'tcn.network.0.net.4.bias', 'tcn.network.0.net.4.weight_g', 'tcn.network.0.net.4.weight_v', 'tcn.network.0.downsample.weight', 'tcn.network.0.downsample.bias', 'tcn.network.1.conv1.bias', 'tcn.network.1.conv1.weight_g', 'tcn.network.1.conv1.weight_v', 'tcn.network.1.conv2.bias', 'tcn.network.1.conv2.weight_g', 'tcn.network.1.conv2.weight_v', 'tcn.network.1.net.0.bias', 'tcn.network.1.net.0.weight_g', 'tcn.network.1.net.0.weight_v', 'tcn.network.1.net.4.bias', 'tcn.network.1.net.4.weight_g', 'tcn.network.1.net.4.weight_v', 'tcn.network.2.conv1.bias', 'tcn.network.2.conv1.weight_g', 'tcn.network.2.conv1.weight_v', 'tcn.network.2.conv2.bias', 'tcn.network.2.conv2.weight_g', 'tcn.network.2.conv2.weight_v', 

In [3]:
num_samples = 0
# fig, ax = plt.subplots(1,1)

curr_im = np.ones((28,28), dtype=np.uint8)*255
_, encoded_image = cv2.imencode('.png', curr_im)
im_bytes = encoded_image.tobytes()
im_disp = widgets.Image(value=im_bytes, width=200, height=200)

true_val = widgets.Label(value=f'True Label: N/A')
pred_val = widgets.Label(value=f'Predicted Label: N/A')
label_disp = widgets.VBox((true_val, pred_val))

display.display(im_disp)
display.display(label_disp)

for data, target in test_loader:
    im = data.squeeze().cpu().detach().numpy()
    rows, cols = im.shape
    im = (im - im.min())
    im = (im/im.max() * 255).astype('uint8')
    curr_im = np.ones(im.shape, dtype=np.uint8)*255
    
    true_val.value = f'True Label: {target.item()}'

    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        num_samples += 1
        curr_row = i // cols
        curr_col = i % cols
        
        curr_im[curr_row,curr_col] = im[curr_row,curr_col]
        _, encoded_image = cv2.imencode('.png', curr_im)
        im_bytes = encoded_image.tobytes()
        im_disp.value = im_bytes
        
    
        output = model.inference(data[:,:,i].view(data.size()[0], data.size()[1], 1))
        pred = output.max(1, keepdim=True)[1] #max returns values and indices
        
        if num_samples > model.receptive_field:
            pred_val.value = f'Predicted Label: {pred.item()}'
    
    time.sleep(2)
    
    curr_im = np.zeros(im.shape, dtype=np.uint8)
    _, encoded_image = cv2.imencode('.png', curr_im)
    im_bytes = encoded_image.tobytes()
    im_disp.value = im_bytes
    true_val.value = f'True Label: N/A'
    for i in np.zeros(np.random.randint(50, 200)):
        num_samples += 1
        output = model.inference(torch.tensor([i], dtype=torch.float).cuda().view(1, 1, 1))
        pred = output.max(1, keepdim=True)[1] #max returns values and indices
        if num_samples > model.receptive_field:
            pred_val.value = f'Predicted Label: {pred.item()}'
        


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\x00\x00\x1c\x08\x00\x00\x00\x00Wf\x80H\x…

KeyboardInterrupt: 

In [3]:
num_samples = 0
# fig, ax = plt.subplots(1,1)

im_queue = [0 for i in range(28*28)]
curr_im = np.array(im_queue, dtype=np.uint8).reshape((28,28))
_, encoded_image = cv2.imencode('.png', curr_im)
im_bytes = encoded_image.tobytes()
im_disp = widgets.Image(value=im_bytes, width=200, height=200)

true_val = widgets.Label(value=f'True Label: N/A')
pred_val = widgets.Label(value=f'Predicted Label: N/A')
label_disp = widgets.VBox((true_val, pred_val))

display.display(im_disp)
display.display(label_disp)

for data, target in test_loader:
    im = data.squeeze().cpu().detach().numpy()
    rows, cols = im.shape
    im = (im - im.min())
    im = (im/im.max() * 255).astype('uint8')
    curr_im = np.ones(im.shape, dtype=np.uint8)*255
    
    true_val.value = f'True Label: {target.item()}'

    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        num_samples += 1
        curr_row = i // cols
        curr_col = i % cols
        
        im_queue.append(im[curr_row,curr_col])
        im_queue = im_queue[1:]
        
        curr_im = np.array(im_queue, dtype=np.uint8).reshape((28,28))
        _, encoded_image = cv2.imencode('.png', curr_im)
        im_bytes = encoded_image.tobytes()
        im_disp.value = im_bytes
        
    
        output = model(data[:,:,i].view(data.size()[0], data.size()[1], 1))
        pred = output.max(1, keepdim=True)[1] #max returns values and indices
        
        if num_samples > model.receptive_field:
            pred_val.value = f'Predicted Label: {pred.item()}'
    
    time.sleep(2)
    
    
    for i in np.zeros(np.random.randint(50, 200)):
        im_queue.append(0)
        im_queue = im_queue[1:]
        curr_im = np.array(im_queue, dtype=np.uint8).reshape((28,28)) 
        _, encoded_image = cv2.imencode('.png', curr_im)
        im_bytes = encoded_image.tobytes()
        im_disp.value = im_bytes
        true_val.value = f'True Label: N/A'
        num_samples += 1
        output = model(torch.tensor([i], dtype=torch.float).cuda().view(1, 1, 1))
        pred = output.max(1, keepdim=True)[1] #max returns values and indices
        if num_samples > model.receptive_field:
            pred_val.value = f'Predicted Label: {pred.item()}'
        


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\x00\x00\x1c\x08\x00\x00\x00\x00Wf\x80H\x…

KeyboardInterrupt: 

In [169]:
test = Image.fromarray((im*255).astype('uint8'),'L')


In [170]:
display.display(test)

In [180]:
_, encoded_image = cv2.imencode('.png', (im*255).astype('uint8'))
im_bytes = encoded_image.tobytes()


widgets.Image(value=im_bytes, width=200, height=200)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\x00\x00\x1c\x08\x00\x00\x00\x00Wf\x80H\x…

In [ ]:
num_samples = 0
# fig, ax = plt.subplots(1,1)

for data, target in test_loader:
    im = data.squeeze().cpu().detach().numpy()
    rows, cols = im.shape
    im = (im - im.min())
    im = im/im.max()
    curr_im = np.ones(im.shape)
    
#     plt.imshow(curr_im, cmap='gray', vmin=0, vmax=1)
#     plt.axis('off')


    true_val = widgets.Label(value=f'True Label: {target.item()}')
    pred_val = widgets.Label(value=f'Predicted Label: N/A')
    label_disp = widgets.VBox((true_val, pred_val))
    display.display(label_disp)
    
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        num_samples += 1
        curr_row = i // cols
        curr_col = i % cols
        
        curr_im[curr_row,curr_col] = im[curr_row,curr_col]
        
    
#         if i % 20 == 0:
#             display.display(plt.gcf())
#             display.clear_output(wait=True)
#             plt.imshow(curr_im, cmap='gray', vmin=0, vmax=1)
#             plt.axis('off')
#             plt.pause(0.001)
        
        output = model.inference(data[:,:,i].view(data.size()[0], data.size()[1], 1))
        pred = output.max(1, keepdim=True)[1] #max returns values and indices
        
        pred_val.value = f'Predicted Label: {pred.item()}'
    
#     display.display(plt.gcf())
#     display.clear_output(wait=True)
#     plt.imshow(curr_im, cmap='gray', vmin=0, vmax=1)
#     plt.axis('off')
#     plt.pause(0.001)

    

    break

In [7]:
torch.tensor([0]*64, dtype=torch.float).view(64,1,1)

tensor([[[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]]

In [2]:
# for i in [0]*np.random.randint(50,200):
#     print (i)

np.random.randint(4)

2

In [38]:
data = torch.rand((64,1, 784))
aug_data = torch.zeros(data.size())
clip_size = np.random.randint(0, 200)
pad_size = np.random.randint(0, 100)

aug_data[:, :, clip_size+pad_size:] = data[:,:,:data.size()[2]-(clip_size+pad_size)]
if clip_size > 0:
    aug_data[:, :, :clip_size] = data[torch.randperm(data.size()[0]), :, -clip_size:]


In [35]:
aug_data[0]

tensor([[5.2512e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 3.6543e-01, 9.9937e-01, 3.2467e-01, 3.4381e-01,
         3.8349e-01, 5.3381e-01, 6.8607e-01, 8.6298e-01, 5.9494e-02, 6.2938e-01,
         5.9301e-02, 2.4135e-01, 7.6662e-01, 3.5607e-01, 5.0892e-01, 5.8481e-02,
         9.3448e-01, 3.2786e-01, 9.8140e-01, 4.6339e-01, 5.2412e-01, 9.4282e-01,
         4.7165e-01, 9.0500e

In [32]:
torch.randperm(data.size()[0])

tensor([41,  6, 52,  2, 46, 11, 48, 32, 56, 37, 62, 51, 33, 44, 12,  7,  1, 50,
        22,  3,  9, 49,  5, 35, 21, 18, 38, 19, 15, 24, 57, 10, 47, 60, 43, 28,
        31, 25, 30, 23, 63, 61, 14, 39, 26, 42, 59,  0, 34, 16,  8, 40, 58, 54,
        13, 17, 53, 27, 45, 36, 55,  4, 29, 20])